# NVIDIA RAG Blueprint - Complete Test Pipeline

This notebook provides a complete end-to-end test of the NVIDIA RAG Blueprint system with:

- ✅ **Unlimited Processing Time**: No timeouts, processes until complete
- ✅ **Real-time Progress Monitoring**: Visual progress indicators and status updates  
- ✅ **Professional Error Handling**: Comprehensive error detection and recovery
- ✅ **Complete Pipeline Testing**: From document upload to query responses

## Prerequisites
- RAG services running (RAG server on 8081, Ingestor on 8082)
- Vector database (Milvus) initialized
- NGC API key configured for cloud embedding models

---

In [50]:
# === IMPORTS AND SETUP ===
import requests
import json
import time
import os
import logging
from typing import Dict, Any, Optional, List, Union
from pathlib import Path

# Async operations setup
try:
    import aiohttp
    import asyncio
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    print("Installing required packages...")
    import subprocess
    subprocess.check_call(["pip", "install", "-q", "aiohttp", "nest_asyncio"])
    import aiohttp
    import asyncio
    import nest_asyncio
    nest_asyncio.apply()

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All imports and async setup complete")

✅ All imports and async setup complete


In [51]:
# === CONFIGURATION ===
class RAGConfig:
    """Centralized configuration for RAG system"""
    
    # Service endpoints
    IPADDRESS = "localhost"
    RAG_PORT = "8081"
    INGESTOR_PORT = "8082"
    
    # URLs
    RAG_BASE_URL = f"http://{IPADDRESS}:{RAG_PORT}"
    INGESTOR_BASE_URL = f"http://{IPADDRESS}:{INGESTOR_PORT}"
    
    # API endpoints  
    RAG_HEALTH_URL = f"{RAG_BASE_URL}/v1/health"
    CHAIN_URL = f"{RAG_BASE_URL}/v1/generate"
    SEARCH_URL = f"{RAG_BASE_URL}/v1/search"
    
    INGESTOR_HEALTH_URL = f"{INGESTOR_BASE_URL}/v1/health"
    DOCUMENTS_URL = f"{INGESTOR_BASE_URL}/v1/documents"
    COLLECTION_URL = f"{INGESTOR_BASE_URL}/v1/collection"
    COLLECTIONS_URL = f"{INGESTOR_BASE_URL}/v1/collections"
    
    # Collection settings
    COLLECTION_NAME = "multimodal_data"
    EMBEDDING_DIMENSION = 2048  # NVIDIA embedding model dimension
    
    # Headers
    HEADERS = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

config = RAGConfig()
print("✅ Configuration loaded")
print(f"   RAG Server: {config.RAG_BASE_URL}")
print(f"   Ingestor: {config.INGESTOR_BASE_URL}")
print(f"   Collection: {config.COLLECTION_NAME}")

✅ Configuration loaded
   RAG Server: http://localhost:8081
   Ingestor: http://localhost:8082
   Collection: multimodal_data


In [52]:
# === HEALTH CHECK FUNCTIONS ===

async def check_service_health(service_name: str, url: str, timeout: int = 10) -> Dict[str, Any]:
    """Check health of a specific service"""
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url, timeout=aiohttp.ClientTimeout(total=timeout)) as response:
                if response.status == 200:
                    result = await response.json()
                    return {"healthy": True, "status": response.status, "details": result}
                else:
                    return {"healthy": False, "status": response.status, "error": "Non-200 status"}
    except asyncio.TimeoutError:
        return {"healthy": False, "error": "Timeout"}
    except Exception as e:
        return {"healthy": False, "error": str(e)}

async def comprehensive_health_check() -> Dict[str, Any]:
    """Perform complete health check on all services"""
    print("🔍 Starting comprehensive health check...")
    
    # Check services
    services = {
        "RAG Server": config.RAG_HEALTH_URL,
        "Ingestor Service": config.INGESTOR_HEALTH_URL
    }
    
    health_results = {}
    for service_name, url in services.items():
        health_results[service_name] = await check_service_health(service_name, url)
    
    # Check vector database
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(config.COLLECTIONS_URL, timeout=aiohttp.ClientTimeout(total=10)) as response:
                if response.status == 200:
                    collections = await response.json()
                    collection_names = [c.get('collection_name', 'unknown') for c in collections.get('collections', [])]
                    health_results["Vector Database"] = {
                        "healthy": True,
                        "collections": collection_names,
                        "target_collection_exists": config.COLLECTION_NAME in collection_names
                    }
                else:
                    health_results["Vector Database"] = {"healthy": False, "error": f"Status {response.status}"}
    except Exception as e:
        health_results["Vector Database"] = {"healthy": False, "error": str(e)}
    
    # Calculate summary
    all_healthy = all(result.get("healthy", False) for result in health_results.values())
    target_collection_exists = health_results.get("Vector Database", {}).get("target_collection_exists", False)
    
    return {
        "overall_healthy": all_healthy,
        "target_collection_exists": target_collection_exists,
        "services": health_results
    }

print("✅ Health check functions defined")

✅ Health check functions defined


In [53]:
# === RUN HEALTH CHECK ===

health_status = await comprehensive_health_check()

print("\n" + "="*60)
print("🏥 HEALTH CHECK RESULTS")
print("="*60)

for service, status in health_status["services"].items():
    emoji = "✅" if status.get("healthy") else "❌"
    print(f"{emoji} {service}: {'Healthy' if status.get('healthy') else 'Unhealthy'}")
    if status.get("error"):
        print(f"   Error: {status['error']}")
    if service == "Vector Database" and status.get("collections"):
        print(f"   Collections: {status['collections']}")

print(f"\n📊 Overall Status: {'✅ All Systems Operational' if health_status['overall_healthy'] else '❌ Issues Detected'}")
print(f"📁 Target Collection: {'✅ Exists' if health_status['target_collection_exists'] else '⚠️ Missing'}")

if not health_status["overall_healthy"]:
    print("\n❌ CRITICAL: Services not healthy - cannot proceed")
    print("   Please start all required services before continuing")
else:
    print("\n✅ All services ready - proceeding to next steps")

print("="*60)

🔍 Starting comprehensive health check...

🏥 HEALTH CHECK RESULTS
✅ RAG Server: Healthy
✅ Ingestor Service: Healthy
✅ Vector Database: Healthy
   Collections: ['multimodal_data', 'metadata_schema', 'test_collection', 'meta']

📊 Overall Status: ✅ All Systems Operational
📁 Target Collection: ✅ Exists

✅ All services ready - proceeding to next steps


In [54]:
# === COLLECTION AND DOCUMENT MANAGEMENT ===

async def create_collection_if_needed(collection_name: str = None) -> bool:
    """Create collection in vector store if it doesn't exist"""
    if collection_name is None:
        collection_name = config.COLLECTION_NAME
    
    data = {
        "collection_name": collection_name,
        "embedding_dimension": config.EMBEDDING_DIMENSION
    }
    
    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(config.COLLECTION_URL, json=data, headers=config.HEADERS) as response:
                if response.status == 200:
                    print(f"✅ Collection '{collection_name}' created successfully!")
                    return True
                else:
                    result = await response.text()
                    if "already exists" in result.lower():
                        print(f"ℹ️  Collection '{collection_name}' already exists")
                        return True
                    else:
                        print(f"⚠️ Failed to create collection: {result}")
                        return False
        except Exception as e:
            print(f"❌ Error creating collection: {e}")
            return False

async def get_document_count(collection_name: str = None) -> int:
    """Get current document count in collection"""
    if collection_name is None:
        collection_name = config.COLLECTION_NAME
    
    try:
        params = {"collection_name": collection_name}
        async with aiohttp.ClientSession() as session:
            async with session.get(config.DOCUMENTS_URL, params=params) as response:
                if response.status == 200:
                    result = await response.json()
                    return result.get('total_documents', 0)
    except Exception as e:
        logger.error(f"Error getting document count: {e}")
    return 0

def create_test_document() -> str:
    """Create a comprehensive test document with facts for RAG testing"""
    
    test_document_content = """# NVIDIA RAG Test Document

## Introduction
This is a comprehensive test document for the NVIDIA RAG Blueprint system. It contains various facts and information designed to test the retrieval and generation capabilities.

## Test Facts

### Geography
- The capital of France is Paris, known for the Eiffel Tower and rich cultural heritage
- Tokyo is the capital of Japan and one of the world's most populous metropolitan areas
- London is the capital of the United Kingdom, located on the River Thames
- New York City is the largest city in the United States by population

### Technology
- Python is a high-level programming language created by Guido van Rossum in 1991
- JavaScript is the programming language of the web, enabling interactive websites
- Docker containers provide isolated environments for running applications consistently
- Kubernetes orchestrates containerized applications across clusters of machines
- Git is a distributed version control system for tracking changes in source code

### Artificial Intelligence
- Machine learning models can process natural language and understand context
- The NVIDIA embedding model produces 2048-dimensional vectors for text representation
- The RTX 5070 Ti is a powerful GPU designed for AI workloads and gaming
- The RTX 4060 is a consumer GPU suitable for moderate AI tasks
- RAG stands for Retrieval Augmented Generation, combining search with AI generation
- Vector embeddings enable semantic search and similarity matching
- Transformer models revolutionized natural language processing since 2017
- BERT and GPT are popular transformer-based architectures
- Milvus is a vector database optimized for storing and searching embeddings

### Computing Concepts
- CPU stands for Central Processing Unit, the brain of the computer
- GPU stands for Graphics Processing Unit, optimized for parallel processing
- RAM provides fast temporary storage for active programs and data
- SSD offers faster storage than traditional hard disk drives
- VRAM is dedicated memory on graphics cards for visual processing
- CUDA enables parallel computing acceleration on NVIDIA GPUs

### Cloud Computing
- Cloud NIMs provide AI models as a service without local hardware requirements
- API endpoints allow remote access to computational resources
- Latency is the delay between request and response in network communications
- Throughput measures the amount of data processed per unit time

## System Architecture
The NVIDIA RAG Blueprint uses a microservices architecture with:
- Ingestion service for document processing
- Embedding service for vector generation
- Vector database for similarity search
- LLM service for response generation
- Reranking service for result optimization

## Performance Metrics
- Default chunk size: 512 tokens
- Chunk overlap: 150 tokens  
- Embedding dimensions: 2048
- Processing time: varies with cloud API response

This document tests the complete RAG pipeline from ingestion to query response.
"""
    
    # Save test document
    test_file_path = "rag_test_document.md"
    with open(test_file_path, 'w', encoding='utf-8') as f:
        f.write(test_document_content)
    
    print(f"✅ Test document created: {test_file_path}")
    print(f"   Size: {len(test_document_content):,} characters")
    print(f"   Location: {os.path.abspath(test_file_path)}")
    
    return test_file_path

print("✅ Collection and document management functions defined")

✅ Collection and document management functions defined


In [55]:
# === SETUP COLLECTION AND DOCUMENT ===

if health_status["overall_healthy"]:
    print(f"📁 Ensuring collection '{config.COLLECTION_NAME}' exists...")
    collection_ready = await create_collection_if_needed(config.COLLECTION_NAME)
    
    if collection_ready:
        initial_doc_count = await get_document_count(config.COLLECTION_NAME)
        print(f"📊 Current documents in collection: {initial_doc_count}")
        
        # Create test document
        print(f"\n📄 Creating test document...")
        test_file_path = create_test_document()
    else:
        print("❌ Failed to create/verify collection")
        collection_ready = False
        test_file_path = None
else:
    print("⚠️ Skipping setup - services not healthy")
    collection_ready = False
    test_file_path = None

📁 Ensuring collection 'multimodal_data' exists...
✅ Collection 'multimodal_data' created successfully!
📊 Current documents in collection: 0

📄 Creating test document...
✅ Test document created: rag_test_document.md
   Size: 2,977 characters
   Location: /home/hongyu/Documents/rag/notebooks/rag_test_document.md


In [56]:
# === UNLIMITED TIME UPLOAD FUNCTIONS ===

async def upload_document_unlimited_time(file_path: str, collection_name: str = None) -> bool:
    """Upload document with UNLIMITED processing time and progress monitoring"""
    if collection_name is None:
        collection_name = config.COLLECTION_NAME
    
    # Check initial state
    initial_count = await get_document_count(collection_name)
    print(f"📊 Initial document count: {initial_count}")
    
    # Prepare upload data
    data = {
        "collection_name": collection_name,
        "blocking": False,  # Use non-blocking for progress monitoring
        "split_options": {
            "chunk_size": 512,
            "chunk_overlap": 150
        }
    }
    
    # Prepare form data
    form_data = aiohttp.FormData()
    
    try:
        file_size = os.path.getsize(file_path)
        with open(file_path, "rb") as f:
            form_data.add_field(
                "documents", 
                f.read(), 
                filename=os.path.basename(file_path),
                content_type="application/octet-stream"
            )
        print(f"📄 File: {os.path.basename(file_path)} ({file_size:,} bytes)")
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        return False
    
    form_data.add_field("data", json.dumps(data), content_type="application/json")
    
    print(f"🚀 Starting upload with UNLIMITED processing time...")
    print(f"   Will monitor progress until completion (no timeout)")
    print(f"   Press Ctrl+C to interrupt if needed\n")
    
    # Upload with NO timeout
    async with aiohttp.ClientSession() as session:
        try:
            # Remove all timeouts - unlimited processing time
            timeout = aiohttp.ClientTimeout(total=None, connect=30)
            async with session.post(
                config.DOCUMENTS_URL, 
                data=form_data,
                timeout=timeout
            ) as response:
                if response.status == 200:
                    result = await response.json()
                    job_id = result.get('job_id')
                    print(f"✅ Upload initiated successfully!")
                    if job_id:
                        print(f"📋 Job ID: {job_id}")
                    
                    # Start unlimited progress monitoring
                    return await monitor_unlimited_progress(collection_name, initial_count, file_path)
                else:
                    text = await response.text()
                    print(f"⚠️ Upload failed with status: {response.status}")
                    print(f"   Response: {text[:200]}...")
                    return False
        except Exception as e:
            print(f"❌ Error uploading document: {e}")
            return False

async def monitor_unlimited_progress(collection_name: str, initial_count: int, file_path: str) -> bool:
    """Monitor processing progress with UNLIMITED time - no timeouts"""
    
    print(f"\n🔄 UNLIMITED PROGRESS MONITORING")
    print(f"   Collection: {collection_name}")
    print(f"   File: {os.path.basename(file_path)}")
    print(f"   Initial documents: {initial_count}")
    print(f"   ⏰ NO TIME LIMIT - will run until completion")
    print(f"   ⌨️  Press Ctrl+C to interrupt if needed\n")
    
    start_time = time.time()
    check_interval = 5  # Check every 5 seconds for better responsiveness
    last_count = initial_count
    
    # Progress animation characters
    progress_chars = ['⠋', '⠙', '⠹', '⠸', '⠼', '⠴', '⠦', '⠧']
    progress_idx = 0
    
    try:
        while True:
            # Get current document count
            current_count = await get_document_count(collection_name)
            elapsed = time.time() - start_time
            minutes = int(elapsed // 60)
            seconds = int(elapsed % 60)
            
            # Animate progress indicator
            progress_char = progress_chars[progress_idx % len(progress_chars)]
            progress_idx += 1
            
            # Show progress line with better time display
            status_msg = ""
            if elapsed < 30:
                status_msg = "Starting processing..."
            elif elapsed < 60:
                status_msg = "Cloud API processing..."
            elif elapsed < 180:
                status_msg = "Still processing (normal for cloud APIs)..."
            else:
                status_msg = f"Long processing ({minutes}m {seconds}s) - cloud APIs can be slow..."
            
            print(f"\r{progress_char} Time: {minutes:2d}m {seconds:2d}s | Docs: {current_count:2d} | {status_msg}", end="", flush=True)
            
            # Check if document count increased
            if current_count > last_count:
                print(f"\n📈 Document count increased! ({last_count} → {current_count})")
                last_count = current_count
                
                # Check if processing completed
                if current_count > initial_count:
                    print(f"\n✅ PROCESSING COMPLETED SUCCESSFULLY!")
                    print(f"   📊 Final document count: {current_count}")
                    print(f"   ⏱️  Total processing time: {minutes}m {seconds}s")
                    print(f"   🎉 Document ready for RAG queries!")
                    return True
            
            # Wait before next check (shorter interval for better responsiveness)
            await asyncio.sleep(check_interval)
            
    except KeyboardInterrupt:
        print(f"\n⚠️ INTERRUPTED by user after {minutes}m {seconds}s")
        final_count = await get_document_count(collection_name)
        success = final_count > initial_count
        print(f"   📊 Final document count: {final_count}")
        print(f"   {'✅ Processing succeeded' if success else '❌ Processing incomplete'}")
        return success
    except Exception as e:
        print(f"\n❌ ERROR during progress monitoring: {e}")
        # Still check if processing succeeded
        try:
            final_count = await get_document_count(collection_name)
            return final_count > initial_count
        except:
            return False

print("✅ Unlimited time upload functions defined")

✅ Unlimited time upload functions defined


In [ ]:
# === UPLOAD DOCUMENT WITH UNLIMITED TIME ===

if health_status["overall_healthy"] and collection_ready and test_file_path:
    print("\n" + "="*80)
    print("🚀 STARTING UNLIMITED TIME DOCUMENT UPLOAD")
    print("="*80)
    
    upload_success = await upload_document_unlimited_time(test_file_path, config.COLLECTION_NAME)
    
    print("\n" + "="*80)
    if upload_success:
        print("🎉 UPLOAD AND PROCESSING COMPLETED SUCCESSFULLY!")
        print("   ✅ Document is now in the knowledge base")
        print("   ✅ Ready for RAG queries")
    else:
        print("❌ UPLOAD/PROCESSING FAILED")
        print("   ⚠️ Document may not be available for queries")
        print("   💡 Check the progress output above for details")
    print("="*80)
else:
    print("\n⚠️ SKIPPING UPLOAD - Prerequisites not met")
    print(f"   Health status: {health_status['overall_healthy']}")
    print(f"   Collection ready: {collection_ready}")
    print(f"   Test file ready: {test_file_path is not None}")
    upload_success = False


🚀 STARTING UNLIMITED TIME DOCUMENT UPLOAD
📊 Initial document count: 0
📄 File: rag_test_document.md (2,977 bytes)
🚀 Starting upload with UNLIMITED processing time...
   Will monitor progress until completion (no timeout)
   Press Ctrl+C to interrupt if needed

✅ Upload initiated successfully!

🔄 UNLIMITED PROGRESS MONITORING
   Collection: multimodal_data
   File: rag_test_document.md
   Initial documents: 0
   ⏰ NO TIME LIMIT - will run until completion
   ⌨️  Press Ctrl+C to interrupt if needed

⠙ Time:  0m 56s | Docs:  0 | Cloud API processing...

In [ ]:
# === RAG QUERY FUNCTIONS ===

def query_rag(question: str, collection_name: str = None) -> Optional[str]:
    """Send query to RAG service"""
    if collection_name is None:
        collection_name = config.COLLECTION_NAME
    
    payload = {
        "messages": [
            {
                "role": "user",
                "content": question
            }
        ],
        "use_knowledge_base": True,
        "collection_names": [collection_name],
        "stream": False,
        "temperature": 0.2,
        "top_p": 0.7,
        "max_tokens": 1024,
        "reranker_top_k": 5,
        "vdb_top_k": 20
    }
    
    try:
        response = requests.post(config.CHAIN_URL, json=payload, headers=config.HEADERS, timeout=30)
        
        if response.status_code == 200:
            # Parse streaming response format
            full_response = ""
            
            for line in response.text.split('\n'):
                if line.startswith('data: '):
                    data_str = line[6:]  # Remove 'data: ' prefix
                    if data_str and data_str != '[DONE]':
                        try:
                            data = json.loads(data_str)
                            choices = data.get('choices', [])
                            if choices:
                                delta = choices[0].get('delta', {})
                                content_chunk = delta.get('content', '')
                                if content_chunk:
                                    full_response += content_chunk
                        except json.JSONDecodeError:
                            continue
            
            return full_response if full_response else "No response generated"
        else:
            return f"Query failed with status {response.status_code}: {response.text[:200]}"
    except Exception as e:
        return f"Error querying RAG: {e}"

async def test_rag_queries() -> bool:
    """Test RAG system with comprehensive queries"""
    
    # First check document count
    doc_count = await get_document_count(config.COLLECTION_NAME)
    print(f"📊 Current documents in knowledge base: {doc_count}")
    
    if doc_count == 0:
        print("❌ No documents in knowledge base!")
        print("   Queries will return generic responses")
        return False
    
    # Test queries that match our document content
    test_queries = [
        "What is Python and when was it created?",
        "What GPUs are mentioned in the document?",
        "What is the capital of France?",
        "Tell me about Docker containers",
        "What does RAG stand for?",
        "How many dimensions does the NVIDIA embedding model produce?",
        "What is Milvus used for?"
    ]
    
    print(f"\n🧪 Testing {len(test_queries)} RAG queries...\n")
    
    successful_queries = 0
    
    for i, query in enumerate(test_queries, 1):
        print(f"📝 Query {i}/{len(test_queries)}: {query}")
        
        response = query_rag(query)
        
        # Check if response contains actual information
        if response and len(response.strip()) > 20:
            # Check for generic "couldn't find" responses
            if ("couldn't find" not in response.lower() and 
                "more context" not in response.lower() and
                "sorry" not in response.lower()[:50] and
                "no information" not in response.lower()):
                successful_queries += 1
                print(f"💬 ✅ Response: {response[:200]}{'...' if len(response) > 200 else ''}")
            else:
                print(f"💬 ⚠️ Generic response: {response[:150]}{'...' if len(response) > 150 else ''}")
        else:
            print(f"💬 ❌ No valid response")
        
        print("-" * 80)
        time.sleep(1)  # Brief pause between queries
    
    # Summary
    success_rate = (successful_queries / len(test_queries)) * 100
    print(f"\n📊 RAG QUERY TEST RESULTS:")
    print(f"   Successful queries: {successful_queries}/{len(test_queries)} ({success_rate:.1f}%)")
    print(f"   Document count: {doc_count}")
    
    if successful_queries > 0:
        print(f"   ✅ RAG system is working correctly!")
        return True
    else:
        print(f"   ❌ RAG system may have issues")
        return False

print("✅ RAG query functions defined")

In [ ]:
# === TEST RAG QUERIES ===

if health_status["overall_healthy"] and upload_success:
    print("\n" + "="*80)
    print("🧪 TESTING RAG QUERY PIPELINE")
    print("="*80)
    
    query_success = await test_rag_queries()
    
    print("\n" + "="*80)
    if query_success:
        print("🎉 RAG PIPELINE TEST COMPLETED SUCCESSFULLY!")
        print("   ✅ Documents processed and stored correctly")
        print("   ✅ Queries returning relevant information")
        print("   ✅ RAG system fully operational")
    else:
        print("⚠️ RAG PIPELINE TEST HAD ISSUES")
        print("   🔧 May need troubleshooting")
    print("="*80)
else:
    print("\n⚠️ SKIPPING QUERY TESTS - Prerequisites not met")
    print(f"   Services healthy: {health_status['overall_healthy']}")
    print(f"   Upload successful: {upload_success}")
    query_success = False

In [ ]:
# === FINAL COMPREHENSIVE SUMMARY ===

print("\n" + "="*90)
print("📋 NVIDIA RAG BLUEPRINT - COMPLETE PIPELINE TEST SUMMARY")
print("="*90)

# Service Health
print(f"🏥 Service Health: {'✅ All Healthy' if health_status['overall_healthy'] else '❌ Issues Detected'}")
for service, status in health_status['services'].items():
    emoji = "✅" if status.get('healthy') else "❌"
    print(f"   {emoji} {service}")

# Collection Status
print(f"\n📁 Collection Status: {'✅ Ready' if collection_ready else '❌ Failed'}")
if collection_ready:
    final_doc_count = await get_document_count(config.COLLECTION_NAME)
    print(f"   📊 Documents in '{config.COLLECTION_NAME}': {final_doc_count}")

# Upload Status
print(f"\n📤 Document Upload: {'✅ Completed' if upload_success else '❌ Failed'}")
if upload_success:
    print(f"   ✅ Processing completed with unlimited time")
    print(f"   ✅ Document ready for queries")

# Query Status  
print(f"\n🧪 RAG Queries: {'✅ Working' if query_success else '❌ Issues' if 'query_success' in locals() else '⏭️ Skipped'}")
if query_success:
    print(f"   ✅ Knowledge base responding correctly")
    print(f"   ✅ RAG pipeline fully operational")

# Overall Status
overall_success = health_status['overall_healthy'] and collection_ready and upload_success and query_success
print(f"\n🎯 Overall Status: {'🎉 COMPLETE SUCCESS' if overall_success else '⚠️ PARTIAL SUCCESS / ISSUES'}")

if overall_success:
    print("\n✨ CONGRATULATIONS! ✨")
    print("Your NVIDIA RAG Blueprint is fully operational:")
    print("   • All services running and healthy")
    print("   • Document processing with unlimited time works")
    print("   • Knowledge base populated and responding")
    print("   • Ready for production workloads")
else:
    print("\n🔧 Next Steps:")
    if not health_status['overall_healthy']:
        print("   1. Start all required services")
    if not collection_ready:
        print("   2. Fix collection creation issues")
    if not upload_success:
        print("   3. Troubleshoot document processing (check cloud API keys)")
    if not query_success:
        print("   4. Debug RAG query pipeline")

print("\n" + "="*90)
print("📝 KEY FEATURES IMPLEMENTED:")
print("   ✅ Unlimited processing time (no timeouts)")
print("   ✅ Real-time progress monitoring with animations")
print("   ✅ Proper error handling and recovery")
print("   ✅ Complete end-to-end pipeline testing")
print("   ✅ Comprehensive health checking")
print("   ✅ Single test document creation (no duplicates)")
print("="*90)

In [ ]:
# === CLEANUP (OPTIONAL) ===

# Uncomment the next lines if you want to clean up the test document
# if test_file_path and os.path.exists(test_file_path):
#     os.remove(test_file_path)
#     print(f"✅ Cleaned up test file: {test_file_path}")

print("✅ Notebook execution complete")
if test_file_path:
    print(f"   Test document preserved: {test_file_path}")
print("   All functions remain available for further testing")